# Lecture 05: Hierarchical Clustering in SAS

In this notebook we will:
1. Define the SAS library
2. Run hierarchical clustering with **Single Linkage**
3. Draw the dendrogram
4. Run hierarchical clustering with **Ward’s Method**
5. Cut the dendrogram into clusters
6. Check cluster sizes and profiles

## 0. Define the library

We tell SAS where the datasets are stored.  
- Students: use the local path (writable).  
- Teacher: has a read-only shared path (commented out).

c：このライブラリの名前（libref）。プログラム内でデータセットを呼ぶときに使う「ニックネーム」。

In [ ]:
/* Path for students */
libname c "C:\Camillo_clabe";

/* Path for teacher (read-only, commented out) */
/* libname c "---\MaterialeDocente\Camillo_clabe"; */

## 1. Single linkage clustering

- Input dataset: `c.unico_1`  
- Method: **Single linkage** (minimum distance between points)  
- Output: `c.tree_single`  
- `var d10_:` → selects all variables with prefix `d10_`  
- `id id` → uses the variable `id` as identifier  “Which variable should I use as the label (identifier) for each observation in the dendrogram or cluster output?”

In [ ]:
proc cluster data=c.unico_1 method=single outtree=c.tree_single;
    var d10_: ;
    id id;
run;

- `proc cluster`：
クラスター分析を行う手順（プロシジャ）を使う、という命令。
→ 「データの似ているグループを自動で見つける分析」。

- `data=c.unico_1`：
使うデータセットの指定。

- `c` はさっきのライブラリ参照（libname c "C:\Camillo_clabe"）で設定したもの。

- `unico_1` は、そのフォルダ内にあるデータセットの名前。
→ つまり、C:\Camillo_clabe\unico_1.sas7bdat というファイルを使っています。

- `method=single`：
クラスタリングの方法を指定しています。
single は「最短距離法（single linkage method）」という方法。
2つのグループの中で「最も近い2点の距離」を基準にしてグループをまとめます。

- `outtree=c.tree_single`：
分析の結果を「ツリー形式」で保存。
`tree_single` という名前のデータセットが、ライブラリ c に作られます。
この結果は後で「デンドログラム（木のような図）」を描くときに使います。

## 2. Draw dendrogram for Single linkage

We use `PROC TREE` to visualize the hierarchical structure.  
Later we can also cut the tree into groups.

In [ ]:
proc tree data=c.tree_single;
    id id;
run;

## 3. Ward’s method clustering

- Method: **Ward** (minimizes within-cluster variance)  
- Output: `c.tree_ward`  
- This usually produces more compact, homogeneous clusters.

In [ ]:
proc cluster data=c.unico_1 method=ward outtree=c.tree_ward;
    var d10_: ;
    id id;
run;

## 4. Draw dendrogram for Ward’s method

Now we display the Ward dendrogram.

In [ ]:
proc tree data=c.tree_ward;
    id id;
run;

## 5. Cutting the dendrogram

Two options:
1. Fix the **number of clusters** (e.g. 3)  
2. Cut at a specific **height** (fusion distance = 7)

In [ ]:
/* Cut into 3 clusters */
proc tree data=c.tree_ward nclusters=3 out=c.clus3;
    id id;
run;

/* Cut at height=7 */
proc tree data=c.tree_ward height=7 out=c.clus_h7;
    id id;
run;

## 6. Quick checks

- `PROC FREQ` → cluster sizes  
- `PROC MEANS` → cluster profiles (means, standard deviations per cluster)

In [ ]:
/* Cluster sizes */
proc freq data=c.clus3;
    tables cluster / nocum;
run;

/* Cluster profiles */
proc means data=c.clus3 mean std maxdec=2;
    class cluster;
    var d10_: ;
run;